In [1]:
import tempfile
import pandas as pd
import numpy as np
import os
from glob import glob
from hypnospy import Wearable
from hypnospy.data import RawProcessing
from hypnospy.analysis import NonWearingDetector, SleepBoudaryDetector
from hypnospy.analysis import PhysicalActivity
from hypnospy import Experiment
from shutil import copyfile
from datetime import datetime, timedelta
from scipy.signal import butter, filtfilt
from sklearn.metrics import mean_squared_error, cohen_kappa_score


In [2]:

def setup_experiment(file_path, start_hour):
    
    # Configure an Experiment
    exp = Experiment()

    # Iterates over a set of files in a directory.
    # Unfortunately, we have to do it manually with RawProcessing because we are modifying the annotations
    for file in glob(file_path):
        pp = RawProcessing(file,
                           device_location="dw",
                           # HR information
                           col_for_hr="mean_hr",
                           # Activity information
                           cols_for_activity=["activity"],
                           is_act_count=True,
                           # Datetime information
                           col_for_datetime="linetime",
                           strftime="%Y-%m-%d %H:%M:%S",
                           # Participant information
                           col_for_pid="mesaid")

        w = Wearable(pp)  # Creates a wearable from a pp object
        # Invert the two_stages flag. Now True means sleeping and False means awake
        w.data["hyp_annotation"] = (w.data["stages"] > 0)
        exp.add_wearable(w)
        exp.set_freq_in_secs(30)
        w.change_start_hour_for_experiment_day(start_hour)
    return exp

In [3]:
file_path = "./data/collection_mesa_hr/*_combined.csv*"
start_hour = 15    

exp = setup_experiment(file_path, start_hour)


In [4]:
nwd = NonWearingDetector(exp)
nwd.fill_no_activity(-0.0001)

sbd = SleepBoudaryDetector(exp)
sbd.detect_sleep_boundaries(strategy="annotation", output_col="hyp_sleep_period_psg",
                                annotation_col="hyp_annotation",
                                annotation_merge_tolerance_in_minutes=300)


/home/palotti/github/HypnosPy/hypnospy/misc.py:22: UserWarning: Day has no sleep period!
  warnings.warn("Day has no sleep period!")
/home/palotti/github/HypnosPy/hypnospy/misc.py:51: UserWarning: Day has no sleep period!
  warnings.warn("Day has no sleep period!")


KeyboardInterrupt: 

In [23]:
sleep_onsets = []
good_pids = []
for w in exp.get_all_wearables():
    first_psg = w.data[w.data["hyp_sleep_period_psg"] == True]
    
    if first_psg.shape[0] > 0:
        first_psg = first_psg.index[0]
        sleep_onset = w.data.iloc[first_psg][w.time_col] - w.data.iloc[0][w.time_col]
        
        if sleep_onset.seconds / 60 >= 30 and sleep_onset.seconds / 60 <= 240:
            print(sleep_onset, w.get_pid())
            good_pids.append(w.get_pid())
        
        sleep_onsets.append(sleep_onset)
    
    else:
        print("Error:", w.get_pid())

0 days 01:24:00 4723
0 days 02:28:30 3239
0 days 01:08:00 2266
0 days 01:12:30 3399
0 days 00:48:00 3533
0 days 01:04:30 3933
0 days 02:31:00 4774
0 days 00:45:30 6132
0 days 01:32:30 5381
0 days 01:55:00 2533
0 days 02:33:00 5559
0 days 02:17:30 5303
0 days 03:54:00 5692
0 days 02:38:00 1391
0 days 01:07:00 5438
0 days 00:51:00 1385
0 days 00:31:00 550
0 days 01:19:00 4227
0 days 00:34:30 1077
0 days 00:38:00 2711
0 days 03:10:30 3216
0 days 02:20:00 6707
0 days 00:46:00 1162
0 days 02:18:00 5907
0 days 01:14:30 3997
0 days 02:11:00 3394
0 days 00:55:30 4450
0 days 01:01:00 3311
0 days 01:13:30 1863
0 days 02:02:00 6727
0 days 01:13:00 6372
0 days 01:53:30 445
0 days 02:58:30 5714
0 days 02:00:30 4994
0 days 03:18:00 2246
0 days 00:43:00 4428
0 days 01:34:00 4411
0 days 00:33:30 5796
0 days 00:32:00 2603
0 days 01:19:30 2676
0 days 02:00:00 407
0 days 01:42:30 5017
0 days 02:01:30 536
0 days 01:49:30 1520
0 days 02:23:30 2638
0 days 01:03:00 3793
0 days 01:32:00 6445
0 days 01:26:00 3

0 days 01:29:00 1862
0 days 02:41:00 2337
0 days 03:35:30 2549
0 days 01:53:00 4003
0 days 02:04:30 3028
0 days 01:14:30 6804
0 days 01:10:30 1809
0 days 00:31:00 5669
0 days 01:44:00 1795
0 days 02:02:00 677
0 days 01:06:00 4759
0 days 00:46:30 1890
0 days 01:37:30 6755
0 days 02:24:30 1620
0 days 01:21:30 5353
0 days 02:13:00 4948
0 days 01:13:00 692
0 days 03:33:00 1041
0 days 00:32:00 860
0 days 02:06:00 4196
0 days 01:05:00 2954
0 days 01:20:30 6115
0 days 01:27:00 2862
0 days 01:55:30 902
0 days 00:43:00 2636
0 days 00:45:00 3347
0 days 02:41:00 1118
0 days 01:00:30 312
0 days 02:20:00 2374
0 days 01:41:00 3354
0 days 01:06:00 3743
0 days 01:08:00 414
0 days 03:22:30 6467
0 days 02:00:00 5697
0 days 01:17:30 6205
0 days 01:07:00 5029
0 days 01:22:30 6015
0 days 00:48:00 3554
0 days 02:01:00 5617
0 days 02:07:30 118
0 days 01:01:30 4383
0 days 01:09:30 1515
0 days 01:42:00 1768
0 days 02:07:30 529
0 days 03:29:30 3506
0 days 01:23:30 4998
0 days 02:51:00 3481
0 days 01:00:30 5149


0 days 03:37:30 5623
0 days 00:40:00 2443
0 days 01:15:30 6417
0 days 01:20:00 443
0 days 03:18:00 3275
0 days 02:08:00 3570
0 days 02:59:00 5677
0 days 02:10:00 1510
0 days 02:42:00 1324
0 days 02:40:30 6610
0 days 00:44:30 6501
0 days 03:15:00 2527
0 days 01:58:00 5259
0 days 03:40:30 5331
0 days 02:50:00 152
0 days 02:49:00 5847
0 days 00:43:30 3541
0 days 01:03:30 4256
0 days 02:19:00 5157
0 days 01:04:00 6416
0 days 00:39:00 801
0 days 01:44:00 1641
0 days 00:44:00 3492
0 days 00:53:30 6693
0 days 00:32:30 1305
0 days 01:23:00 1672
0 days 02:30:00 4576
0 days 01:25:30 6663
0 days 02:41:00 3818
0 days 01:55:00 6754
0 days 01:15:30 5457
0 days 02:10:00 2217
0 days 01:58:30 6450
0 days 01:48:30 5550
0 days 01:51:30 3043
0 days 00:59:00 2040
0 days 01:28:00 4276
0 days 01:19:30 363
0 days 01:25:30 1707
0 days 01:36:30 1732
0 days 01:23:30 4839
0 days 01:24:00 3537
0 days 02:45:00 5423
0 days 02:02:00 636
0 days 02:20:00 5033
0 days 01:39:30 2183
0 days 03:16:00 1177
0 days 01:41:00 38

In [38]:
outfile_path = "./data/collection_mesa_hr_30_240/"
   
for file in glob(file_path):
    pp = RawProcessing(file,
                       device_location="dw",
                       # HR information
                       col_for_hr="mean_hr",
                       # Activity information
                       cols_for_activity=["activity"],
                       is_act_count=True,
                       # Datetime information
                       col_for_datetime="linetime",
                       strftime="%Y-%m-%d %H:%M:%S",
                       # Participant information
                       col_for_pid="mesaid")
    
    if str(pp.get_pid()) in good_pids:
        w = Wearable(pp)
        df = w.data[["activity", "linetime", "mean_hr", "stages"]].copy()
        df["linetime"] = pd.to_datetime(df["linetime"]) 
        df.set_index("linetime").resample("30s").mean().interpolate()
        df["mesaid"] = w.get_pid()
        df.to_csv(os.path.join(outfile_path, os.path.basename(file)), index=False)
        #copyfile(file, os.path.join(outfile_path, os.path.basename(file)))


In [ ]:
(pd.Series(sleep_onsets).astype('timedelta64[s]') / 60).plot.hist(bins=100)

In [59]:
# Diary

mesa = pd.read_csv("./mesa-sleep-dataset-0.3.0.csv")

# Remove pids without diary
pids_without_diary = set(mesa[mesa["bedtmwkday5c"].isnull()]["mesaid"].unique())
pids_without_diary = pids_without_diary.union(set(mesa[mesa["bedtmwkend5c"].isnull()]["mesaid"].unique()))
pids_without_diary = pids_without_diary.union(set(mesa[mesa["waketmwkday5c"].isnull()]["mesaid"].unique()))
pids_without_diary = pids_without_diary.union(set(mesa[mesa["waketmwkend5c"].isnull()]["mesaid"].unique()))

mesa = mesa[~mesa["mesaid"].isin(pids_without_diary)]


def get_delta(str_time):
    return timedelta(hours=int(str_time.split(":")[0]), 
                     minutes=int(str_time.split(":")[1]),
                     seconds=int(str_time.split(":")[2]))


def create_diary_for_mesa(pid, onset_weekday, onset_weekend, offset_weekday, offset_weekend, sleepdur_weekday, sleepdur_weekend):
    
    rows = []
    onset_shift = 0
    offset_shift = 0
    
    for d in range(1, 15):
        row = {}
        row["pid"] = pid
        
        day = datetime.strptime("2017-01-%02d" % (d), '%Y-%m-%d')
#         if day.weekday() in [4, 5]:  # Monday=0, Sunday=6.
#             onset_shift = 1 if onset_weekend[0] == '0' else 0
#             onset = datetime.strptime("2017-01-%02d %s" % (d+onset_shift, onset_weekend), '%Y-%m-%d %H:%M:%S')
#         else:
#             onset_shift = 1 if onset_weekday[0] == '0' else 0
#             onset = datetime.strptime("2017-01-%02d %s" % (d+onset_shift, onset_weekday), '%Y-%m-%d %H:%M:%S')            
#         row["sleep_onset"] = onset
        
#         offset_shift = 0 if onset.hour < 10 else 1
        
#         if day.weekday() in [5, 6]: # Monday=0, Sunday=6.
#             offset = datetime.strptime("2017-01-%02d %s" % (d+onset_shift+offset_shift, offset_weekend), '%Y-%m-%d %H:%M:%S')
#         else:
#             offset = datetime.strptime("2017-01-%02d %s" % (d+onset_shift+offset_shift, offset_weekday), '%Y-%m-%d %H:%M:%S')
#         row["sleep_offset"] = offset


        if day.weekday() in [4, 5]:  # Monday=0, Sunday=6.
            onset_shift = 1 if onset_weekend[0] == '0' else 0
            onset = datetime.strptime("2017-01-%02d %s" % (d + onset_shift, onset_weekend), '%Y-%m-%d %H:%M:%S')
            offset = onset + get_delta(sleepdur_weekend)
        else:
            onset_shift = 1 if onset_weekday[0] == '0' else 0
            onset = datetime.strptime("2017-01-%02d %s" % (d, onset_weekday), '%Y-%m-%d %H:%M:%S')
            offset = onset + get_delta(sleepdur_weekday)
        
        row["sleep_onset"] = onset
        row["sleep_offset"] = offset
        rows.append(row)
    return rows

df = []
for rid, row in mesa.iterrows():
    print(rid)
    df.extend(create_diary_for_mesa(*(row[["mesaid", "bedtmwkday5c", "bedtmwkend5c", "waketmwkday5c", "waketmwkend5c", "wkdaysleepdur5c", "wkendsleepdur5c"]])))
    
pd.DataFrame(df).sort_values(by=["pid","sleep_onset"]).to_csv("data/diaries/mesa_diary.csv", index=False)
#df = pd.DataFrame(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
62
63
64
65
66
67
68
69
70
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
147
148
149
150
151
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283

2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
